# Week 05: Tree 심화 & BST - 실습 노트북

---

## 🔧 필수 라이브러리 Import

In [ ]:
# 표준 라이브러리
import sys
from typing import List, Optional, Tuple, Dict, Set
import time

# 자료구조 관련
from collections import deque, defaultdict, Counter
from heapq import heappush, heappop, heapify
from bisect import bisect_left, bisect_right, insort

# 알고리즘 관련
from functools import lru_cache, reduce
from itertools import permutations, combinations, product, accumulate
import math

print("Python version:", sys.version)
print("라이브러리 import 완료!")

## 🎯 Section 1: 핵심 개념 구현

### 1.1 TreeNode 클래스 정의

In [ ]:
# 기본 TreeNode 클래스
class TreeNode:
    def __init__(self, val=0, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right
    
    def __repr__(self):
        return f"TreeNode({self.val})"

# 트리 생성 헬퍼 함수
def build_tree(values):
    """리스트로부터 이진 트리 생성 (레벨 순서)"""
    if not values:
        return None
    
    root = TreeNode(values[0])
    queue = deque([root])
    i = 1
    
    while queue and i < len(values):
        node = queue.popleft()
        
        # 왼쪽 자식
        if i < len(values) and values[i] is not None:
            node.left = TreeNode(values[i])
            queue.append(node.left)
        i += 1
        
        # 오른쪽 자식
        if i < len(values) and values[i] is not None:
            node.right = TreeNode(values[i])
            queue.append(node.right)
        i += 1
    
    return root

# 트리 시각화 함수
def print_tree(root, level=0, prefix="Root: "):
    """트리를 시각적으로 출력"""
    if root:
        print(" " * (level * 4) + prefix + str(root.val))
        if root.left or root.right:
            if root.left:
                print_tree(root.left, level + 1, "L--- ")
            else:
                print(" " * ((level + 1) * 4) + "L--- None")
            if root.right:
                print_tree(root.right, level + 1, "R--- ")
            else:
                print(" " * ((level + 1) * 4) + "R--- None")

# 테스트
print("=== 트리 생성 테스트 ===")
test_tree = build_tree([1, 2, 3, 4, 5, 6, 7])
print_tree(test_tree)

### 1.2 Top-down vs Bottom-up DFS 비교

In [ ]:
# Top-down 접근: 부모에서 자식으로 정보 전달
def max_depth_topdown(root):
    """Top-down 방식으로 트리 깊이 계산
    시간복잡도: O(n)
    공간복잡도: O(h)
    """
    def dfs(node, depth):
        if not node:
            return depth
        
        # 현재 깊이를 자식에게 전달
        left_depth = dfs(node.left, depth + 1)
        right_depth = dfs(node.right, depth + 1)
        
        return max(left_depth, right_depth)
    
    return dfs(root, 0)

# Bottom-up 접근: 자식에서 부모로 정보 수집
def max_depth_bottomup(root):
    """Bottom-up 방식으로 트리 깊이 계산
    시간복잡도: O(n)
    공간복잡도: O(h)
    """
    if not root:
        return 0
    
    # 자식들의 결과를 먼저 얻음
    left_depth = max_depth_bottomup(root.left)
    right_depth = max_depth_bottomup(root.right)
    
    # 결과를 이용해 현재 노드 계산
    return 1 + max(left_depth, right_depth)

# 테스트
print("=== Top-down vs Bottom-up 비교 ===")
test_tree = build_tree([1, 2, 3, 4, 5, None, 6, 7])
print("트리 구조:")
print_tree(test_tree)
print(f"\nTop-down 깊이: {max_depth_topdown(test_tree)}")
print(f"Bottom-up 깊이: {max_depth_bottomup(test_tree)}")

### 1.3 BST 구현 및 연산

In [ ]:
# BST 노드 클래스
class BSTNode:
    def __init__(self, val=0, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right
    
    def __repr__(self):
        return f"BSTNode({self.val})"

# BST 클래스
class BST:
    def __init__(self):
        self.root = None
    
    def insert(self, val):
        """BST에 값 삽입"""
        if not self.root:
            self.root = BSTNode(val)
            return
        
        current = self.root
        while True:
            if val < current.val:
                if current.left:
                    current = current.left
                else:
                    current.left = BSTNode(val)
                    break
            else:
                if current.right:
                    current = current.right
                else:
                    current.right = BSTNode(val)
                    break
    
    def search(self, val):
        """BST에서 값 검색"""
        current = self.root
        while current:
            if current.val == val:
                return True
            elif val < current.val:
                current = current.left
            else:
                current = current.right
        return False
    
    def inorder(self):
        """중위 순회 (정렬된 순서)"""
        result = []
        
        def dfs(node):
            if not node:
                return
            dfs(node.left)
            result.append(node.val)
            dfs(node.right)
        
        dfs(self.root)
        return result
    
    def delete(self, val):
        """BST에서 노드 삭제"""
        def delete_node(node, val):
            if not node:
                return None
            
            if val < node.val:
                node.left = delete_node(node.left, val)
            elif val > node.val:
                node.right = delete_node(node.right, val)
            else:
                # 삭제할 노드를 찾음
                if not node.left:
                    return node.right
                if not node.right:
                    return node.left
                
                # 후속자 찾기 (오른쪽 서브트리의 최솟값)
                min_node = node.right
                while min_node.left:
                    min_node = min_node.left
                
                node.val = min_node.val
                node.right = delete_node(node.right, min_node.val)
            
            return node
        
        self.root = delete_node(self.root, val)

# 테스트
print("=== BST 연산 테스트 ===")
bst = BST()
values = [50, 30, 70, 20, 40, 60, 80]

print("삽입할 값들:", values)
for val in values:
    bst.insert(val)

print("중위 순회 (정렬된 순서):", bst.inorder())

print("\n검색 테스트:")
print(f"40 존재? {bst.search(40)}")
print(f"45 존재? {bst.search(45)}")

print("\n30 삭제 후:")
bst.delete(30)
print("중위 순회:", bst.inorder())

## 💡 Section 2: 주요 패턴 실습

### 2.1 패턴 1: 경로 탐색 패턴

In [ ]:
# 패턴 1: 모든 루트-리프 경로 찾기
def all_paths(root):
    """모든 루트-리프 경로 반환
    시간복잡도: O(n × h) - 최악의 경우
    공간복잡도: O(h)
    """
    def dfs(node, path, paths):
        if not node:
            return
        
        # 현재 노드를 경로에 추가
        path.append(node.val)
        
        # 리프 노드인 경우
        if not node.left and not node.right:
            paths.append(path[:])
        else:
            # 계속 탐색
            dfs(node.left, path, paths)
            dfs(node.right, path, paths)
        
        # 백트래킹
        path.pop()
    
    paths = []
    dfs(root, [], paths)
    return paths

# 경로 합이 목표값인 경로 찾기
def path_sum(root, target_sum):
    """목표 합을 만족하는 모든 경로 찾기"""
    def dfs(node, remaining, path, paths):
        if not node:
            return
        
        path.append(node.val)
        remaining -= node.val
        
        # 리프 노드에서 목표값 확인
        if not node.left and not node.right and remaining == 0:
            paths.append(path[:])
        
        dfs(node.left, remaining, path, paths)
        dfs(node.right, remaining, path, paths)
        
        path.pop()
    
    paths = []
    dfs(root, target_sum, [], paths)
    return paths

# 테스트
print("=== 경로 탐색 패턴 ===")
test_tree = build_tree([5, 4, 8, 11, None, 13, 4, 7, 2, None, None, 5, 1])
print("트리 구조:")
print_tree(test_tree)

print("\n모든 경로:")
for path in all_paths(test_tree):
    print(f"  {' -> '.join(map(str, path))}")

print("\n합이 22인 경로:")
for path in path_sum(test_tree, 22):
    print(f"  {' -> '.join(map(str, path))} = 22")

### 2.2 패턴 2: 메모이제이션 활용 패턴

In [ ]:
# 패턴 2: 트리 DP with 메모이제이션
def house_robber_iii(root):
    """트리에서 인접하지 않은 노드의 최대 합
    시간복잡도: O(n)
    공간복잡도: O(n) - 캐시
    """
    @lru_cache(maxsize=None)
    def rob_helper(node_id):
        # node_id를 사용해 노드 찾기 (실제로는 node 객체를 직접 사용)
        node = node_map.get(node_id)
        if not node:
            return 0, 0  # (rob_root, not_rob_root)
        
        left_rob, left_not_rob = rob_helper(id(node.left) if node.left else None)
        right_rob, right_not_rob = rob_helper(id(node.right) if node.right else None)
        
        # 현재 노드를 털 경우
        rob_root = node.val + left_not_rob + right_not_rob
        
        # 현재 노드를 털지 않을 경우
        not_rob_root = max(left_rob, left_not_rob) + max(right_rob, right_not_rob)
        
        return rob_root, not_rob_root
    
    # 노드 맵 생성 (id로 노드 찾기)
    node_map = {}
    def build_map(node):
        if node:
            node_map[id(node)] = node
            build_map(node.left)
            build_map(node.right)
    
    build_map(root)
    return max(rob_helper(id(root) if root else None))

# 서브트리 합 빈도수 (메모이제이션 없이)
def most_frequent_subtree_sum(root):
    """가장 빈번한 서브트리 합 찾기"""
    freq = defaultdict(int)
    
    def get_sum(node):
        if not node:
            return 0
        
        # 현재 서브트리의 합 계산
        subtree_sum = node.val + get_sum(node.left) + get_sum(node.right)
        freq[subtree_sum] += 1
        
        return subtree_sum
    
    get_sum(root)
    
    if not freq:
        return []
    
    max_freq = max(freq.values())
    return [sum_val for sum_val, f in freq.items() if f == max_freq]

# 테스트
print("=== 메모이제이션 패턴 ===")

# House Robber III 테스트
rob_tree = build_tree([3, 2, 3, None, 3, None, 1])
print("House Robber III 트리:")
print_tree(rob_tree)
print(f"최대 도둑질 금액: {house_robber_iii(rob_tree)}")

# 서브트리 합 테스트
sum_tree = build_tree([5, 2, -3])
print("\n서브트리 합 트리:")
print_tree(sum_tree)
print(f"가장 빈번한 서브트리 합: {most_frequent_subtree_sum(sum_tree)}")

## 🔥 Section 3: LeetCode 문제 풀이

### Problem 1: [98] Validate Binary Search Tree

In [ ]:
"""
문제 설명:
주어진 이진 트리가 유효한 이진 탐색 트리(BST)인지 판별하라.

BST 조건:
- 노드의 왼쪽 서브트리에는 노드의 값보다 작은 값들만 있어야 함
- 노드의 오른쪽 서브트리에는 노드의 값보다 큰 값들만 있어야 함
- 모든 서브트리도 BST여야 함

예시:
Input: [2,1,3]
Output: true

Input: [5,1,4,null,null,3,6]
Output: false (루트 5의 오른쪽 서브트리에 3이 있음)
"""

class Solution:
    def isValidBST(self, root: Optional[TreeNode]) -> bool:
        """
        접근법:
        1. 각 노드가 가질 수 있는 값의 범위를 전달
        2. 왼쪽 자식은 (min, parent.val) 범위
        3. 오른쪽 자식은 (parent.val, max) 범위
        
        시간복잡도: O(n)
        공간복잡도: O(h)
        """
        def validate(node, min_val, max_val):
            if not node:
                return True
            
            # 현재 노드가 범위를 벗어나면 False
            if node.val <= min_val or node.val >= max_val:
                return False
            
            # 왼쪽과 오른쪽 서브트리 검증
            return (validate(node.left, min_val, node.val) and
                   validate(node.right, node.val, max_val))
        
        return validate(root, float('-inf'), float('inf'))
    
    def isValidBST_inorder(self, root: Optional[TreeNode]) -> bool:
        """
        대체 접근법: 중위 순회가 오름차순인지 확인
        """
        def inorder(node):
            if not node:
                return []
            return inorder(node.left) + [node.val] + inorder(node.right)
        
        values = inorder(root)
        for i in range(1, len(values)):
            if values[i] <= values[i-1]:
                return False
        return True

# 테스트
solution = Solution()
test_cases = [
    ([2, 1, 3], True),
    ([5, 1, 4, None, None, 3, 6], False),
    ([1], True),
    ([10, 5, 15, None, None, 6, 20], False),
]

for i, (values, expected) in enumerate(test_cases, 1):
    tree = build_tree(values)
    result = solution.isValidBST(tree)
    print(f"테스트 {i}: {'✅' if result == expected else '❌'}")
    print(f"  입력: {values}")
    print(f"  출력: {result}")
    print(f"  예상: {expected}\n")

### Problem 2: [230] Kth Smallest Element in a BST

In [ ]:
"""
문제 설명:
BST가 주어졌을 때, k번째로 작은 원소를 찾아라.

예시:
Input: root = [3,1,4,null,2], k = 1
Output: 1

Input: root = [5,3,6,2,4,null,null,1], k = 3
Output: 3
"""

class Solution:
    def kthSmallest(self, root: Optional[TreeNode], k: int) -> int:
        """
        접근법 1: 중위 순회로 k번째 원소 찾기
        시간복잡도: O(h + k)
        공간복잡도: O(h)
        """
        def inorder(node):
            if node:
                # 제너레이터로 메모리 효율적 구현
                yield from inorder(node.left)
                yield node.val
                yield from inorder(node.right)
        
        # k번째 원소까지만 순회
        gen = inorder(root)
        for _ in range(k - 1):
            next(gen)
        return next(gen)
    
    def kthSmallest_iterative(self, root: Optional[TreeNode], k: int) -> int:
        """
        접근법 2: 반복문으로 구현
        """
        stack = []
        current = root
        count = 0
        
        while stack or current:
            # 왼쪽 끝까지 이동
            while current:
                stack.append(current)
                current = current.left
            
            # 노드 처리
            current = stack.pop()
            count += 1
            
            if count == k:
                return current.val
            
            # 오른쪽으로 이동
            current = current.right
        
        return -1

# 테스트
solution = Solution()
test_cases = [
    ([3, 1, 4, None, 2], 1, 1),
    ([5, 3, 6, 2, 4, None, None, 1], 3, 3),
    ([3, 1, 4, None, 2], 3, 3),
]

for i, (values, k, expected) in enumerate(test_cases, 1):
    tree = build_tree(values)
    result = solution.kthSmallest(tree, k)
    print(f"테스트 {i}: {'✅' if result == expected else '❌'}")
    print(f"  트리: {values}")
    print(f"  k = {k}")
    print(f"  출력: {result}")
    print(f"  예상: {expected}\n")

### Problem 3: [235] Lowest Common Ancestor of a BST

In [ ]:
"""
문제 설명:
BST에서 두 노드의 최소 공통 조상(LCA)을 찾아라.

예시:
Input: root = [6,2,8,0,4,7,9,null,null,3,5], p = 2, q = 8
Output: 6

Input: root = [6,2,8,0,4,7,9,null,null,3,5], p = 2, q = 4
Output: 2
"""

class Solution:
    def lowestCommonAncestor(self, root: TreeNode, p: TreeNode, q: TreeNode) -> TreeNode:
        """
        접근법: BST 속성 활용
        - p와 q가 모두 현재 노드보다 작으면 왼쪽으로
        - p와 q가 모두 현재 노드보다 크면 오른쪽으로
        - 그 외의 경우 현재 노드가 LCA
        
        시간복잡도: O(h)
        공간복잡도: O(1)
        """
        current = root
        
        while current:
            # 둘 다 왼쪽에 있음
            if p.val < current.val and q.val < current.val:
                current = current.left
            # 둘 다 오른쪽에 있음
            elif p.val > current.val and q.val > current.val:
                current = current.right
            # 갈라지는 지점 (또는 하나가 현재 노드)
            else:
                return current
        
        return None
    
    def lowestCommonAncestor_recursive(self, root: TreeNode, p: TreeNode, q: TreeNode) -> TreeNode:
        """
        재귀 버전
        """
        if p.val < root.val and q.val < root.val:
            return self.lowestCommonAncestor_recursive(root.left, p, q)
        elif p.val > root.val and q.val > root.val:
            return self.lowestCommonAncestor_recursive(root.right, p, q)
        else:
            return root

# 테스트를 위한 도우미 함수
def find_node(root, val):
    """트리에서 특정 값을 가진 노드 찾기"""
    if not root:
        return None
    if root.val == val:
        return root
    left = find_node(root.left, val)
    if left:
        return left
    return find_node(root.right, val)

# 테스트
solution = Solution()
test_tree = build_tree([6, 2, 8, 0, 4, 7, 9, None, None, 3, 5])
print("BST 구조:")
print_tree(test_tree)

test_cases = [
    (2, 8, 6),
    (2, 4, 2),
    (0, 5, 2),
]

print("\nLCA 테스트:")
for p_val, q_val, expected in test_cases:
    p = find_node(test_tree, p_val)
    q = find_node(test_tree, q_val)
    result = solution.lowestCommonAncestor(test_tree, p, q)
    print(f"LCA({p_val}, {q_val}) = {result.val if result else None} {'✅' if result and result.val == expected else '❌'}")

## 📝 Section 4: 핵심 정리 및 팁

### 이번 주 핵심 포인트

#### 1. **Tree DFS 심화의 핵심**
- Top-down: 부모에서 자식으로 정보 전달 (깊이, 경로 등)
- Bottom-up: 자식에서 부모로 정보 수집 (높이, 합계 등)
- 경로 문제는 백트래킹이 핵심

#### 2. **BST의 핵심**
- 중위 순회 = 정렬된 순서
- 검색/삽입/삭제 모두 O(h) 시간복잡도
- BST 속성을 활용한 가지치기가 중요

#### 3. **자주 하는 실수**
- ❌ BST 검증 시 직접 자식만 비교 (전체 서브트리 확인 필요)
- ❌ 경로 문제에서 백트래킹 누락
- ✅ 올바른 방법: 범위 전달 또는 중위 순회 활용

#### 4. **Python 꿀팁**
```python
# 제너레이터로 메모리 효율적인 순회
def inorder(node):
    if node:
        yield from inorder(node.left)
        yield node.val
        yield from inorder(node.right)

# @lru_cache로 트리 DP 최적화
@lru_cache(maxsize=None)
def tree_dp(node_id):
    # 메모이제이션으로 중복 계산 방지
    pass

# nonlocal로 외부 변수 수정
def tree_problem():
    max_val = 0
    def dfs(node):
        nonlocal max_val
        max_val = max(max_val, node.val)
```